<a href="https://colab.research.google.com/github/BoringName1234/Assignment-1/blob/main/BB84_Protocol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Start by installing qiskit and checking the install

In [3]:
!pip install qiskit qiskit-aer pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=ec517abf4a333e36d051798b615ade3eb38e5d7b5606afeed86410018cc172df
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc


In [4]:
import qiskit
print("Qiskit version:", qiskit.__version__)

Qiskit version: 2.2.3


We need a custom implementation of the Most Frequent Count function, that doesn't crash when there are multiple equally frequent counts


In [37]:
def most_frequent(counts):
  """Return the most frequent count

  Returns:
      str: The bit string for the most frequent result. If there are multiple most frequent results, return the first.
  """

  max_value = max(counts.values())
  max_values_counts = [x[0] for x in counts.items() if x[1] == max_value]

  return max_values_counts[0]

This class defines each agent in the protocol.

In [41]:
import random

class Agent:
  """
  An agent in the BB84 protocol. This class contains the agent's gate choices, the type of agent, and the filtered keys.
  """
  def __init__(self, num_bits, agent_type):

    self.num_bits = num_bits
    self.agent_type = agent_type
    self.key = []
    self.sifted_key = []
    self.shared_secret_key = []
    self.bases = []

    if(agent_type == "Sender"):
      self.generate_key(num_bits)



  def generate_key(self, num_bits):
    """
    Generates the key to be transmitted. We will be randomly setting the bit values.
    """
    self.key = [random.randint(0,1) for i in range(20)]

  def define_base_choices(self, num_bits):
    """
    Defines the gates that the agent will send or recieve with. This will be done randomly.
    A 0 represents the Z-basis with |0⟩ → 0,|1⟩ → 1, 1 represents the X-basis |+⟩ → 0, |−⟩ → 1.
    """
    self.bases = [random.randint(0,1) for i in range(20)]


We can now define the circut that represents the quantum transmissions between two parties.
If there is an eavesdropper, then we must make a circuit connecting Alice and Eve, then Eve and Bob, because there is no way to handle Eve's mid transmission measurements in Qiskit.

In [ ]:
def Transmission_Circut(num_qubits, sender, reciever):
  """
  Defines the quantum circut that simulates the transmission of between a sender and reciever in BB84.
  The reciever is either Bob or Eve; if the reciever is Eve, then another circuit must be made to simulate
  Eve forwarding the qubits to Bob.
  :param num_qubits: The number of qubits to transmit.
  :param sender: The agent that transmits the qubits.
  :param reciever: The agent that recieves the qubits.

  :return: A QuantumCircuit object.
  """

  qc = QuantumCircuit(num_qubits)

  # Start by adding the sender's measurement gates
  for i, bit in enumerate(sender.key):
    # If the bit is 1, apply NOT gate to transfer to
    if bit == 1:
      qc.x(i)
    # Then apply the Hadamard gate if the bit is transmitted in the X basis
    if sender.bases[i] == 1:
      qc.h(i)

  # Add the reciever's measurement gates
  for i in range (num_qubits):
    if(reciever.bases[i] == 1):
      qc.h(i)

  # The measurement projects each qubit into the Z-basis and reads them
  qc.measure_all()

  return qc

**OLD CODE**



In [104]:
from qiskit import QuantumCircuit

def BB84_Circuit(num_qubits, alice, bob, eve=None):
  """
  Defines the quantum circut that simulates the transmission of qubits.
  :param num_qubits: The number of qubits to transmit.
  :param alice: The agent that transmits the qubits.
  :param bob: The agent that recieves the qubits.
  :param eve: The eavesdropper, which is set to None by default.

  :return: A QuantumCircuit object.
  """

  qc = QuantumCircuit(num_qubits)

  # Start by adding gates that a
  for i, bit in enumerate(alice.key):
    # If the bit is 1, apply NOT gate to transfer to
    if bit == 1:
      qc.x(i)
    # Then apply the Hadamard gate if the bit is transmitted in the X basis
    if alice.bases[i] == 1:
      qc.h(i)

  # Add Eve's Hadamard gates if there is an eavesdropper
  if eve != None:
    for i in range (num_qubits):
      # Eve measures in a random base, and then transmits in the same bases
      if eve.bases[i] == 1:
        qc.h(i)
        qc.measure(i)
        qc.h(i)
      else:
        qc.measure(i)

  # Add Bob's measurement gates
  for i in range (num_qubits):
    if(bob.bases[i] == 1):
      qc.h(i)

  # The measurement projects each qubit into the Z-basis and reads them
  qc.measure_all()

  return qc

Now we can define the simulation

In [107]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer, AerSimulator
from qiskit import transpile

def BB84_Simulation(num_bits, n, confidence_threshold=1, noise=False, eavesdropper=False):
  """
  Runs an instance of the BB84 protocol.

  :param num_bits: The bit length of the key we want to transmit.
  :param n: The number of bits that will be publically revealed.
  :param confidence_threshold: The confidence threshold for detecting the eavesdropper.
  :param noise: States if the simulation should include noise.
  :param eavesdropper: States if the simulation should include an eavesdropper.
  :return: None
  """

  # We start by defining alice and bob, with the key length and type
  alice = Agent(num_bits, "Sender")
  alice.define_base_choices(num_bits)

  bob = Agent(num_bits, "Receiver")
  bob.define_base_choices(num_bits)

  if(eavesdropper == False):
    #qc = BB84_Circuit(num_bits, alice, bob)
    qc = Transmission_Circut(num_bits, alice, bob)
  else:
    eve = Agent(num_bits, "Eavesdropper")
    eve.define_base_choices(num_bits)

    qc = BB84_Circuit(num_bits, alice, bob, eve)

  if(noise == False):
    noise_level = 0
    #backend = Aer.get_backend('qasm_simulator')
  #else:
    #TODO
    #print("Unimplemented")

  backend = Aer.get_backend('qasm_simulator')
  new_cirq = transpile(qc, backend)
  job = backend.run(new_cirq)
  counts = job.result().get_counts()

  # Retrieve the most common result of the circut
  measured_bits = most_frequent(counts)
  # Then reverse the bitsring, as the circut measures backwards
  measured_bits = measured_bits[::-1]
  bob.key = [int(x) for x in str(measured_bits)]

  # This is the sifting stage:
  # Sender announces their bases, and reciever compares
  # If the bases match, then they will keep the bit, otherwise it is discarded
  num_correct = 0
  for i in range (num_bits):
    if alice.bases[i] == bob.bases[i]:
      alice.sifted_key.append(alice.key[i])
      bob.sifted_key.append(bob.key[i])

      # Eve will keep bits that Alice and Bob keep
      if(eavesdropper == True):
        eve.sifted_key.append(eve.key[i])

  # If there are no matching bases, then the simulation must be restarted
  if(len(alice.sifted_key) == 0):
    print("Alice and Bob did not have any matching bases.")
    return
  elif(len(alice.sifted_key) <= n):
    print("There are too few bits to perform the eavesdropper check.")
    return

  # Alice and Bob randomly select a portion of their bits to share and compare
  index = [i for i in range(len(alice.sifted_key))]
  comparison_index = random.sample(index, n)
  same_key_bits = 0
  for i in comparison_index:
    if alice.sifted_key[i] == bob.sifted_key[i]:
      same_key_bits += 1

  # The remaining bits are the secret key
  for i in range(len(alice.sifted_key)):
    if (i not in comparison_index):
      alice.shared_secret_key.append(alice.sifted_key[i])
      bob.shared_secret_key.append(bob.sifted_key[i])

      # Eve doesn't know if every bit is correct, but they will keep the key
      if(eavesdropper == True):
        eve.shared_secret_key.append(eve.sifted_key[i])

  success_rate = same_key_bits/n
  print("Estimated probability of each bit in final key being correct: ", success_rate)

  if(success_rate < confidence_threshold):
    print("Eavesdropper detected, so the key is discarded.")
  else:
    print("No eavesdropper detected.")
    if(alice.shared_secret_key == bob.shared_secret_key):
      print("Alice and Bob have the same shared secret key: ", alice.shared_secret_key)
    else:
      print("Alice and Bob have different secret keys.")
      print("Alice's key: ", alice.shared_secret_key)
      print("Bob's key: ", bob.shared_secret_key)

    if(eavesdropper == True):
      print("There was an eavesdropper, who believes the key is: ", eve.shared_secret_key)



This is a 20 bit simulation, with 5 comparison bits, no eavesdropper, and no noise.

In [108]:
BB84_Simulation(20,5)

ValueError: invalid literal for int() with base 10: ' '

This is a 20 bit simulation with 5 comparison bits, an eavesdropper, and no noise.

In [94]:
BB84_Simulation(20,5, eavesdropper=True)

TypeError: QuantumCircuit.measure() missing 1 required positional argument: 'cbit'